In [1]:
## general libraries 
import os,sys,glob
import numpy as np
import pandas as pd
import xarray as xr
import h5py

# for plots
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-whitegrid')

print(f'Using Python  = {sys.version.split()[0]}')

Using Python  = 3.8.8


In [2]:
from itertools import chain

#### We will use the following coordinates:
- latitude = 84
- longitude = 17.5

##### Upload datasets 

In [3]:
path = '//storage/ddiaz/2023/myProject/data/'
file_sl = 'era5_monthly_avg_sl_2005-2020_tisr_tciw_tclw_tcwv_hcc_mcc_lcc_sp_tco3_fal_tsr.nc'
file_pl= 'era5_monthly_avg_pl_2015-2020_cc_ciwc_clwc.nc'

In [4]:
xrr_sl = xr.open_dataset(path+file_sl)

xrr_sl_odd = xrr_sl.sel(time = ['2015','2017','2019'])
# even years for training
xrr_sl_even = xrr_sl.sel(time = ['2016','2018','2020'])

In [5]:
df_sl_train = xrr_sl_even.to_dataframe()
#df_sl_test = xrr_sl_odd.to_dataframe()

In [6]:
#To convert to W m-2 , the accumulated values should be divided by the accumulation period expressed in seconds
df_sl_train.tisr = df_sl_train.tisr/24/3600
df_sl_train.tsr =df_sl_train.tsr/24/3600

#df_sl_test.tisr = df_sl_test.tisr/24/3600
#df_sl_test.tsr =df_sl_test.tsr/24/3600

In [7]:
xrr_pl = xr.open_dataset(path+file_pl)

#xrr_pl_odd = xrr_pl.sel(time = ['2015','2017','2019'])
xrr_pl_even = xrr_pl.sel(time = ['2016','2018','2020'])

In [8]:
df_pl_train = xrr_pl_even.to_dataframe()
#df_pl_test = xrr_pl_odd.to_dataframe()

### Variables

In [9]:
ciwc_train = np.array(df_pl_train.ciwc) #kg kg-1
clwc_train = np.array(df_pl_train.clwc) #kg kg-1

#### single levels

In [10]:
p = xrr_pl.level.values

#### pressure levels

In [11]:
cc = np.array(df_pl_train.cc)

In [12]:
dp = []
for ii in np.arange(len(p)-1):
    DP = p[ii+1]-p[ii]
    dp.append(DP)

In [13]:
#We add one value to dP so every level of pressure has a dP value
dp.insert(0,100)   

In [14]:
g=9.8

In [15]:
data_arrays = []
for ii in np.arange(18):
    pL = p[ii]
    DP = dp[ii]
    xrr = xrr_pl.sel(level = pL)
    cwp =100*1000*(xrr.ciwc+xrr.clwc)*DP/g #g/m2
    data_arrays.append(cwp)

In [16]:
xrr_cwp = xr.concat(data_arrays,dim='level')#xrr_cwp.combine(cwp)

##### Total cloud water path (CWP)

tot_cwp = (lambda ii:sum(ii))(cwp[0])
tot_cwp #g/m2